In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import heartpy as hp #이 모듈은 깔아야 import 할수 있음
import seaborn as sns
import scipy.stats as ss
import pywt

from sklearn.preprocessing import minmax_scale 

In [2]:
colname = ['ppg_grn', 'SKT', 'GSR', 'valence', 'arousal']
#data = pd.read_csv('WMinMaxs2e5.csv', header = None)
#data.columns = colname

MotherWavelet_ppg = pywt.Wavelet('db2')   # Mother wavelet (모함수) 지정
MotherWavelet_gsr = pywt.Wavelet('db3')

select  = 4                    # 특징추출 영역 고주파 영역부터 개수 지정 (d1~)

# 함수 정의
def rms(x):
    return np.sqrt(np.mean(x**2))

def crestF(x):
    return np.max(x) / rms(x)

def shapeF(x):
    return rms(x) / np.mean(x)

def impulseF(x):
    return np.max(x)/ np.mean(x)

def zero_out_detailed(coeff, level):
    for i in range(1, level+1):
        for j in range(0, len(coeff[i])):
            coeff[i][j] = 0

# Phasic = GSR - A10 
def get_phasic_amp_and_max(gsr_raw, recon_a10):
    phasic = []
    domain_size = min(len(gsr_raw), len(recon_a10))
    for i in range(0, domain_size): 
        phasic.append(gsr_raw[i] - recon_a10[i])
    # find amplitude and max 
    amp = max(phasic[0:domain_size]) - min(phasic[0:domain_size])
    max_val = max(phasic[0:domain_size])
    return amp, max_val

# SCVSR = A8 - A10
def get_scvsr_zcross(recon_a8, recon_a10):  
    A8minusA10 = [] 
    domain_size = min(len(recon_a8), len(recon_a10))
    for i in range(0, domain_size): 
        A8minusA10.append(recon_a8[i] - recon_a10[i])
    # normalize 
    norm_scvsr = minmax_scale(A8minusA10)
    # consider offset 
    for i in range(0, len(norm_scvsr)):
        norm_scvsr[i] = norm_scvsr[i] - 0.5
    # find zero cross   
    zero_cross_scvsr = np.nonzero(np.diff(norm_scvsr > 0))[0]
    return zero_cross_scvsr

# SCSR = A6 - A10
def get_scsr_zcross(recon_a6, recon_a10): 
    A6minusA10 = []
    domain_size = min(len(recon_a6), len(recon_a10))
    for i in range(0, domain_size):
        A6minusA10.append(recon_a6[i] - recon_a10[i])
    # normalize 
    norm_scsr = minmax_scale(A6minusA10)
    # consider offset 
    for i in range(0, len(norm_scsr)):
        norm_scsr[i] = norm_scsr[i] - 0.5 
    #find zero cross 
    zero_cross_scsr = np.nonzero(np.diff(norm_scsr > 0))[0]
    return zero_cross_scsr



In [3]:
# 결과값 딕셔너리로 반환 함수
def NN_dictionary(working_data, time_interval, data1, strd):
    jump = int(time_interval/strd)
    RR_list = working_data['RR_list']
    RR_diff = working_data['RR_diff']
    cum = np.cumsum(RR_list)
    time = list(range(0, int(cum[-1]), strd))

    #PPG 시간영역(0~5)
    meanNN = []
    medianNN = []
    SDNN = []
    pNN50 = []
    NN50 = []
    RMSSD = []
    
    #PPG 주파수영역(6~16)
    maxFPPG = []
    minFPPG = []
    meanFPPG = []
    stdFPPG = []
    maxminFPPG = []
    kurFPPG = []
    skewFPPG = []
    rmsFPPG = []
    crestFPPG = []
    shapeFPPG = []
    impulseFPPG = []
    

    #GSR 시간영역(17~27)
    maxGSR = []
    minGSR = []
    meanGSR = []
    stdGSR = []
    maxminGSR =[]
    kurGSR = []
    skewGSR = []
    rmsGSR = []
    crestGSR = []
    shapeGSR = []
    impulseGSR = []
    
    
    #GSR 주파수영역(28-31)
    zeroCrossScvsrFGSR = []
    zeroCrossScsrFGSR = []
    phasicMaxAmpFGSR = []
    phasicMaxValFGSR = []
    
    #SKT 시간영역(31~41)
    maxSKT = []
    minSKT = []
    meanSKT = []
    stdSKT = []
    maxminSKT = []
    kurSKT = []
    skewSKT = []
    rmsSKT = []
    crestSKT = []
    shapeSKT = []
    impulseSKT = []

    #valence arousal labeling(41~44)
    label_valence = []
    label_arousal = []
    valence = []
    arousal = []

    
    for i in range(0, len(time) - jump):
        #Extract data in a range
        data1 = data.iloc[int(i*fs*time_interval/1000) : int((i+1)*fs*time_interval/1000), 0:5]
        if(data1.shape == (0, 5)):
            continue
        
        print(data1.shape)
        idx = (cum >= time[i]) & (cum < time[i + jump])
        NN = RR_list[idx]
        diff = []
        for j in range(0, len(NN) - 1):
            diff.append(NN[j + 1] - NN[j])

        num50 = sum(np.array(diff) > 50)
        if len(diff) > 0:
            p50 = num50 / len(diff)
        else:
            p50 = 0
        rmssd = np.sqrt(np.mean(sum(NN ** 2)))

        meanNN.append(np.mean(NN))
        medianNN.append(np.median(NN))
        SDNN.append(np.std(NN))
        NN50.append(num50)
        pNN50.append(p50)
        RMSSD.append(rmssd)
    
        LevelFour = 4
        PPGCoef = pywt.wavedec(data1['ppg_grn'], MotherWavelet_ppg, level=LevelFour, axis=0)
        ppgApprCoef = PPGCoef[LevelFour]
    
        # GSR Frequency Domain Analysis
        LevelTen = 10
        GSRCoefTen = pywt.wavedec(data1['GSR'], MotherWavelet_gsr, level=LevelTen, axis = 0)
        zero_out_detailed(GSRCoefTen, 10)
        recon_ten = pywt.waverec(GSRCoefTen, 'db3')

        LevelEight = 8 
        GSRCoefEight = pywt.wavedec(data1['GSR'], MotherWavelet_gsr, level=LevelEight, axis = 0)
        zero_out_detailed(GSRCoefTen, 8)
        recon_eight = pywt.waverec(GSRCoefEight, 'db3')

        LevelSix = 6
        GSRCoefSix = pywt.wavedec(data1['GSR'], MotherWavelet_gsr, level=LevelSix, axis = 0)
        zero_out_detailed(GSRCoefSix, 6)
        recon_six = pywt.waverec(GSRCoefSix, 'db3')
        
        # Frequency Domain PPG 
        maxFPPG.append(np.max(ppgApprCoef))
        minFPPG.append(np.min(ppgApprCoef))
        meanFPPG.append(np.mean(ppgApprCoef))
        stdFPPG.append(np.std(ppgApprCoef))
        maxminFPPG.append(np.ptp(ppgApprCoef))
        kurFPPG.append(ss.kurtosis(ppgApprCoef))
        skewFPPG.append(ss.skew(ppgApprCoef))
        rmsFPPG.append(rms(ppgApprCoef))
        crestFPPG.append(crestF(ppgApprCoef))
        shapeFPPG.append(shapeF(ppgApprCoef))
        impulseFPPG.append(impulseF(ppgApprCoef))
        
        # Time Domain GSR
        maxGSR.append(np.max(data1['GSR']))
        minGSR.append(np.min(data1['GSR']))
        meanGSR.append(np.mean(data1['GSR']))
        stdGSR.append(np.std(data1['GSR']))
        maxminGSR.append(np.ptp(data1['GSR']))
        kurGSR.append(ss.kurtosis(data1['GSR']))
        skewGSR.append(ss.skew(data1['GSR']))
        rmsGSR.append(rms(data1['GSR']))
        crestGSR.append(crestF(data1['GSR']))
        shapeGSR.append(shapeF(data1['GSR']))
        impulseGSR.append(impulseF(data1['GSR']))
        
        # Frequency Domain GSR
        amp_FGSR, val_FGSR = get_phasic_amp_and_max(data1['GSR'], recon_ten)
        zeroCrossScvsrFGSR.append(get_scvsr_zcross(recon_eight, recon_ten))       # SCVSR = A8 - A10
        zeroCrossScsrFGSR.append(get_scsr_zcross(recon_eight, recon_ten))         # SCSR = A6 - A10
        phasicMaxAmpFGSR.append(amp_FGSR)          # Phasic = GSR - A10 
        phasicMaxValFGSR.append(val_FGSR)          # Phasic = GSR - A10 
        
        # Time Domain Skin Temperatrue
        maxSKT.append(np.max(data1['SKT']))
        minSKT.append(np.min(data1['SKT']))
        meanSKT.append(np.mean(data1['SKT']))
        stdSKT.append(np.std(data1['SKT']))
        maxminSKT.append(np.ptp(data1['SKT']))
        kurSKT.append(ss.kurtosis(data1['SKT']))
        skewSKT.append(ss.skew(data1['SKT']))
        rmsSKT.append(rms(data1['SKT']))
        crestSKT.append(crestF(data1['SKT']))
        shapeSKT.append(shapeF(data1['SKT']))
        impulseSKT.append(impulseF(data1['SKT']))
        
        # Emotion Label
        valence.append(np.mean(data1['valence']))
        arousal.append(np.mean(data1['arousal']))
        label_valence.append(int(np.mean(data1['valence'])*3/10.01)-1)
        label_arousal.append(int(np.mean(data1['arousal'])*3/10.01)-1)

    result = {'meanNN': meanNN,
              'medianNN': medianNN,
              'SDNN': SDNN,
              'pNN50': pNN50,
              'NN50': NN50,
              'RMSSD': RMSSD,  # ,'HR':HR

              'maxFPPG': maxFPPG,
              'minFPPG': minFPPG,
              'meanFPPG': meanFPPG,
              'stdFPPG': stdFPPG,
              'maxminFPPG': maxminFPPG,
              'kurFPPG': kurFPPG,
              'skewFPPG': skewFPPG,
              'rmsFPPG': rmsFPPG,
              'crestFPPG': crestFPPG,
              'shapeFPPG': shapeFPPG,
              'impulseFPPG': impulseFPPG,

              'maxGSR': maxGSR,
              'minGSR': minGSR,
              'meanGSR': meanGSR,
              'stdGSR': stdGSR,
              'maxminGSR': maxminGSR,
              'kurGSR': kurGSR,
              'skewGSR': skewGSR,
              'rmsGSR': rmsGSR,
              'crestGSR': crestGSR,
              'shapeGSR': shapeGSR,
              'impulseGSR': impulseGSR,
              
              'zeroCrossScvsrFGSR': zeroCrossScvsrFGSR,
              'zeroCrossScsrFGSR': zeroCrossScsrFGSR,
              'phasicAmpFGSR': phasicMaxAmpFGSR,
              'phasicMaxFGSR': phasicMaxValFGSR, 

              'maxSKT': maxSKT,
              'minSKT': minSKT,
              'meanSKT': meanSKT,
              'stdSKT': stdSKT,
              'maxminSKT': maxminSKT,
              'kurSKT': kurSKT,
              'skewSKT': skewSKT,
              'rmsSKT': rmsSKT,
              'crestSKT': crestSKT,
              'shapeSKT': shapeSKT,
              'impulseSKT': impulseSKT,

              'valence': valence,
              'arousal': arousal,
              'label_valence': label_valence,
              'label_arousal': label_arousal

              }

    return result
    


In [4]:
"""
working_data, measures = hp.process(data['ppg_grn'].values, fs, calc_freq=True)
result = pd.DataFrame(NN_dictionary(working_data, time_interval, data))
print(result)
"""
#result.to_csv('timefeaturetest.csv', index = False)
#sns.pairplot(result)
#plt.show()

fs = 398
time_interval = 11940
strd = 400

Dir = 'C:/Users/Jihwan/Desktop/Test_Data' #경로 설정
outputDir = 'C:/Users/Jihwan/Desktop/Processed/'
os.chdir(Dir)
filelist = os.listdir(Dir) #Dir 경로 폴더 안 모든 파일 이름 리스트

#idx=0;

for file in filelist:
    data = pd.read_csv(file, header = None)
    data.columns = colname
    working_data, measures = hp.process(data['ppg_grn'].values, fs, calc_freq=False)
    result = pd.DataFrame(NN_dictionary(working_data, time_interval, data, strd))
    result.to_csv(outputDir + file, index = False)


(4752, 5)
(4752, 5)


C:\Users\Jihwan\anaconda3\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 10 is too high: all coefficients will experience boundary effects.
  warnings.warn(
C:\Users\Jihwan\anaconda3\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 10 is too high: all coefficients will experience boundary effects.
  warnings.warn(


KeyError: 0